In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time


import warnings
warnings.filterwarnings('ignore')

In [ ]:
def scroll(driver):
    driver.maximize_window()
    time.sleep(3)
    time_to_wait = 20 
    start = time.time()
    while True:
        driver.execute_script('scrollBy(0,400)')
        time.sleep(0.5)
        if start + time_to_wait < time.time():
            break
        
    try:
        button=driver.find_element(by =By.CSS_SELECTOR,value =f'[data-stid="show-more-results"]')
        button.click()
        time.sleep(3)
    except:
        text_box_list = driver.find_elements(by = By.XPATH , value='.//*[@class="uitk-spacing uitk-spacing-margin-blockstart-three"]')
        return text_box_list

    text_box_list = driver.find_elements(by = By.XPATH , value='.//*[@class="uitk-spacing uitk-spacing-margin-blockstart-three"]')
    return text_box_list

In [ ]:
def single_search(url, driver, Snapshot_date, checking_date_type, checkout_date_type):
    

    hotels = []
    index = 1
    driver.get(url)  
    text_boxes_list = scroll(driver)
    print(len(text_boxes_list))
    for t_box in text_boxes_list:
        try:
            hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,
                     "Num of Reviews":None,"Curr Price":None,"Original Price":None,"Distance from center":None,
                     "Type of room":None,"Location grade":None ,"Extra Data":None}
            
            name = t_box.find_element(by= By.CLASS_NAME , value="uitk-heading").text
            score = t_box.find_element(By.CLASS_NAME, 'uitk-spacing-padding-inlineend-half').text
            price = t_box.find_element(By.CSS_SELECTOR, "div[data-test-id='price-summary-message-line']").text.split('$')
          
            hotel['Snapshot'] = Snapshot_date
            hotel['Index'] = index
            hotel['Hotel Name'] = name
            hotel['TTT'] = (checking_date_type - Snapshot_date).days
            hotel['LOS'] = (checkout_date_type - checking_date_type).days
            hotel['Grade'] = score 
            hotel['Curr Price'] = price[-1]
            if len(price) > 3:
                hotel['Original Price'] = price[1].split('\n')[0]
            hotels.append(hotel)
            
            index = index + 1
        except:
             print('error')
    
    return pd.DataFrame.from_dict(hotels)

In [ ]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe", 'yair':'C:\Windows\chromdriver.exe' }
driver = webdriver.Chrome(path['roy'])

In [ ]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)

print(combinations_list , len(combinations_list)) 

In [ ]:
#chack how much hotels we can collect
'''
city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
LIST__ = []
X= 0
for comb in combinations_list:
    if X==15:
        break
    X =X +1
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
    driver.get(url)
    LIST__.append(len(scroll(driver)))
    

print(LIST__)'''

In [ ]:
# for dev
combinations_list = [combinations_list[0]]
# END - for dev



city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
expedia_df = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Grade","Num of Reviews","Curr Price","Original Price",
                                   "Distance from center","Type of room","Location grade", "Extra Data"])
for comb in combinations_list:
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
    single_search_df  = single_search(url, driver , Snapshot_date, checking_date_type, checkout_date_type)
    expedia_df = pd.concat([expedia_df, single_search_df])
    print(single_search_df)
#driver.quit()

In [ ]:
expedia_df

In [ ]:
len(expedia_df)

--------------------------------------------------------------------------------------------------------------------------------------------

TRY HERE:


In [ ]:
path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': r"C:\Users\yarin\chromedriver.exe", 'yair':'C:\Windows\chromdriver.exe' }
driver = webdriver.Chrome(path['roy'])
city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
checking_year = 2023
checking_month = 3
checking_monthday = 5
checkout_year= 2023
checkout_month= 3
checkout_monthday= 8
comb = ['2023-03-10','2023-03-12']
url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
driver.get(url)



text_boxes_list = scroll(driver)


print(len(text_boxes_list))

In [ ]:

# extract values from card:
x=0
for t_box in text_boxes_list:
 
    try:
        name = t_box.find_element(by= By.CLASS_NAME , value="uitk-heading").text
        score = t_box.find_element(By.CLASS_NAME, 'uitk-spacing-padding-inlineend-half').text
        price = t_box.find_element(By.CSS_SELECTOR, "div[data-test-id='price-summary-message-line']").text.split('$')
        curr_price = price[-1]
        if len(price) > 3:
            original_price = price[1].split('\n')[0]
        else:
            original_price =curr_price
       
        #extra = t_box.find_element(By.CLASS_NAME, value="uitk-layout-flex-item").text
        #reviews = t_box.find_element(By.CLASS_NAME, value="uitk-layout-flex-item").text
        print(name , "  ",score,"  " , "  ", reviews , "  "  , curr_price , "  ",original_price)
    except:
        print("errror")